Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking, Optim
#gr(size=(600,600));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.24

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);
df2 = filter(row -> row[:age] >= 18, df);
first(df2, 5)

,height,weight,age,male
,Float64,Float64,Float64,Int64
1,151.765,47.8256,63.0,1
2,139.7,36.4858,63.0,0
3,136.525,31.8648,65.0,0
4,156.845,53.0419,41.0,1
5,145.415,41.2769,51.0,0


### snippet 4.25

Our first model:

In [4]:
m4_1 = "
  height ~ Normal(μ, σ) # likelihood
  μ ~ Normal(178,20) # prior
  σ ~ Uniform(0, 50) # prior
"

"\n  height ~ Normal(μ, σ) # likelihood\n  μ ~ Normal(178,20) # prior\n  σ ~ Uniform(0, 50) # prior\n"

### snippet 4.26

Compute MAP

In [5]:
obs = df2[:height]

function loglik(x)
  ll = 0.0
  ll += log(pdf(Normal(178, 20), x[1]))
  ll += log(pdf(Uniform(0, 50), x[2]))
  ll += sum(log.(pdf.(Normal(x[1], x[2]), obs)))
  -ll
end

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:1
└ @ Core string:1


loglik (generic function with 1 method)

### snippet 4.28

In [6]:
x0 = [ 178, 10.0]
lower = [0.0, 0.0]
upper = [250.0, 50.0]

2-element Array{Float64,1}:
 250.0
  50.0

### snippet 4.27

In [7]:
inner_optimizer = GradientDescent()

optimize(loglik, lower, upper, x0, Fminbox(inner_optimizer))

 * Status: success

 * Candidate solution
    Minimizer: [1.55e+02, 7.73e+00]
    Minimum:   1.227916e+03

 * Found with
    Algorithm:     Fminbox with Gradient Descent
    Initial Point: [1.78e+02, 1.00e+01]

 * Convergence measures
    |x - x'|               = 1.69e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.09e-10 ≰ 0.0e+00
    |f(x) - f(x')|         = 4.55e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 3.70e-16 ≰ 0.0e+00
    |g(x)|                 = 4.86e-09 ≤ 1.0e-08

 * Work counters
    Iterations:    4
    f(x) calls:    250
    ∇f(x) calls:   250


Our second model:

In [8]:
m4_2 = "
  height ~ Normal(μ, σ) # likelihood
  μ ~ Normal(178, 0.1) # prior
  σ ~ Uniform(0, 50) # prior
"

"\n  height ~ Normal(μ, σ) # likelihood\n  μ ~ Normal(178, 0.1) # prior\n  σ ~ Uniform(0, 50) # prior\n"

### snippet 4.29

Compute MAP

In [9]:
obs = df2[:height]

function loglik2(x)
  ll = 0.0
  ll += log(pdf(Normal(178, 0.1), x[1]))
  ll += log(pdf(Uniform(0, 50), x[2]))
  ll += sum(log.(pdf.(Normal(x[1], x[2]), obs)))
  -ll
end

optimize(loglik2, lower, upper, x0, Fminbox(inner_optimizer))

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:1
└ @ Core string:1


 * Status: success

 * Candidate solution
    Minimizer: [1.78e+02, 2.45e+01]
    Minimum:   1.629108e+03

 * Found with
    Algorithm:     Fminbox with Gradient Descent
    Initial Point: [1.78e+02, 1.00e+01]

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 5.28e-06 ≰ 1.0e-08

 * Work counters
    Iterations:    6
    f(x) calls:    5207
    ∇f(x) calls:   5207


End of `clip-24-29s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*